In [ ]:
# Visualize the first two reaction centers of a cluster to see if they are actually the same
# This code snippet fails atm, clusters and reaction centers are missing

import matplotlib.pyplot as plt
from synutility.SynVis.graph_visualizer import GraphVisualizer

clusters = None

first_cluster = next(iter(clusters.values()))

rc1 = first_cluster[0]['RC']
rc2 = first_cluster[1]['RC']

fig, ax = plt.subplots(2, 1, figsize=(15, 10))
vis = GraphVisualizer()

# Plot ITS graph and its reaction center
vis.plot_its(rc1, ax[0], use_edge_color=True)
vis.plot_its(rc2, ax[1], use_edge_color=True)

# Display the plots
plt.show()

In [1]:
# WP 4 (Version B)
# Manual implementation of the Weisfeiler Lehmann clustering
# This code snippet fails atm, reaction centers are missing

from synutility.SynIO.data_type import load

data = load('data/ITS_graphs.pkl.gz')

NUM_ITERATIONS = 5

node_hash_table = dict()
next_table_hash = 2

hash_clusters = dict()

# Step 1: Set base hash to 1

for reaction in data:

    G = reaction['RC']

    for node in G.nodes():
        G.nodes[node][f'hash_0'] = 1

hash_clusters['iteration_0'] = {
    '1': data
}

# Step 2: 

for i in range(1, NUM_ITERATIONS + 1):

    # Setup iteration cluster

    hash_clusters[f'iteration_{i}'] = dict()

    previous_iteration_hash_clusters = hash_clusters[f'iteration_{i - 1}']
    iteration_hash_clusters = hash_clusters[f'iteration_{i}']

    for cluster in previous_iteration_hash_clusters.values():

        # Calculate hashes:
        for reaction in cluster:

            G = reaction['RC']

            for node, node_data in G.nodes(data=True):

                hash = node_data.get(f'hash_{i - 1}')
                neighbor_hashes = []

                for neighbor in G.neighbors(node):
                    neighbor_hashes.append(G.nodes[neighbor][f'hash_{i - 1}'])

                neighbor_hashes_tuple = tuple(neighbor_hashes)

                new_hash_key = (hash, neighbor_hashes_tuple)

                if new_hash_key not in node_hash_table:
                    node_hash_table[new_hash_key] = next_table_hash
                    next_table_hash += 1

                G.nodes[node][f'hash_{i}'] = node_hash_table[new_hash_key]

                # Cluster:
        for reaction in cluster:

            G = reaction['RC']

            hash_multiset = tuple(sorted([node_data[f'hash_{i}'] for _, node_data in G.nodes(data=True)]))

            if hash_multiset not in iteration_hash_clusters:

                iteration_hash_clusters[hash_multiset] = [reaction]

            else:

                iteration_hash_clusters[hash_multiset].append(reaction)

for i in range(1, NUM_ITERATIONS + 1):
    print(len(hash_clusters[f'iteration_{i}']))